In [96]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [168]:
def dateDiff(date1, date2):
    if date1 > date2:
        difDays = datetime.datetime.strptime(date1, "%Y-%m-%d") - datetime.datetime.strptime(date2, "%Y-%m-%d")
    else:
        difDays = datetime.datetime.strptime(date2, "%Y-%m-%d") - datetime.datetime.strptime(date1, "%Y-%m-%d")
    return difDays.days

In [28]:
dateDiff("2018-07-09", "2018-07-09")

0

In [14]:
BASE_PATH = "~/Downloads/kaggle/paipaidai/data/"
train_data = pd.read_csv(BASE_PATH + "train.csv", na_values="\\N")
test_data = pd.read_csv(BASE_PATH + "test.csv", na_values="\\N")

In [15]:
train_data.shape, test_data.shape

((1000000, 7), (130000, 5))

In [17]:
train_data['repay_amt'].isnull().sum()

117192

In [65]:
sub_train_data = train_data[~ train_data.repay_date.isnull()]

In [41]:
sub_train_data.dtypes

user_id            int64
listing_id         int64
auditing_date     object
due_date          object
due_amt          float64
repay_date        object
repay_amt        float64
dtype: object

In [66]:
sub_train_data['diff_date'] = sub_train_data.apply(lambda row:dateDiff(row['due_date'], row['repay_date']), axis=1)
sub_train_data['diff_amt'] = sub_train_data.apply(lambda row:row['repay_amt'] - row['due_amt'], axis=1)

/Users/lkjin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/lkjin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [76]:
def audit_order(sub_train_data):
    sub_train_data = sub_train_data.sort_values(['user_id', 'listing_id'])
    sub_train_data['group_rank'] = sub_train_data[["user_id", "listing_id"]].groupby(["user_id"]).rank(method="min")
    return sub_train_data

In [193]:
test_min = test_data.listing_id.min()
train_max = sub_train_data.listing_id.max()
data = pd.concat([sub_train_data[list(test_data.columns)], test_data])
new_data = audit_order(data)
new_data.shape

(1012808, 6)

In [194]:
listing_info = pd.read_csv(BASE_PATH + "listing_info.csv", na_values="\\N")
new_data = new_data.merge(listing_info, on=['user_id', 'listing_id', 'auditing_date'], how="inner")
new_data.shape

(1012808, 9)

In [196]:
user_info = pd.read_csv(BASE_PATH + "user_info.csv", na_values="\\N")
new_data = new_data.merge(user_info, on=["user_id"], how="left")

new_data['diff_days'] = new_data.apply(lambda row: dateDiff(row['auditing_date'], row['insertdate']), axis=1)
new_data['diff_min'] = new_data.groupby("listing_id")['diff_days'].transform("min")

new_data = new_data[(new_data['diff_days'] == new_data['diff_min'])].sort_values(['listing_id', 'insertdate']).drop_duplicates("listing_id")
new_data.shape

(1012808, 18)

In [157]:
user_info['user_id'].nunique()

928195